# RAAGs as subgroups of Out(RAAGs)

by Manuel Wiedmer, September 2022

The following code contains some functions that can be used to do computations involving the outer automorphism groups of right-angled groups. In particular, they can be used to find examples of right-angled Artin groups as subgroups of the outer automorphism groups of a right-angled Artin group. Among other things, one can construct for any given right-angled Artin group A_Lambda with at least three vertices a graph Gamma such that A_Lambda is a finite-index subgroup of Out(A_Gamma). Part of this code is based on the code from Benjamin Brück used for [A note on virtual duality and automorphism groups of right-angled Artin groups](https://www.cambridge.org/core/journals/glasgow-mathematical-journal/article/note-on-virtual-duality-and-automorphism-groups-of-rightangled-artin-groups/361EEE6AD7E12AC5F7D87DE26CFFBF13), which can be found on https://github.com/benjaminbrueck/computations_for_roars. In particular, the following functions (in the file "functions.py") were taken from that code: ```star```, ```conn_comps```, ```is_Sil```, ```supp_graph``` and ```theta```, where the last function was slightly adapted.

The code was used for [Right-angled Artin groups as finite-index subgroups of their outer automorphism groups](https://arxiv.org/abs/2209.02033), which is a shortened version of my [Master Thesis](https://www.research-collection.ethz.ch/handle/20.500.11850/538688).

First, one needs to import the necessary modules:

In [ ]:
import networkx as nx
import itertools as it
import copy
import matplotlib.pyplot as plt
import random
import sys
import math
import pickle
import os
from functions import *
import time
import cProfile

%matplotlib inline

In general, for saving graphs we used the pickle module, which allows to save Python objects internally in a .txt-file so that one can reuse them
later in Python. For example in the function random_graphs below we check at the beginning whether there is already a file with the name name_of_output_file. If so, we load the already found pairs of graphs. In general, all files with "\_internal" need to be made readable before opening them in a text editor. This can be done with the function "print_file" from "functions.py". Also, when we later talk of the size of a graph, we mean its number of vertices.

With the following code, we used the condition by Day-Wade to check for all graphs Gamma of a fixed size n whether PSO(A_Gamma) is a right-angled Artin group. We used this for all n up to 7; for larger n the computation took too long to finish. One could change this code to also check whether PSO(A_Gamma) has finite index in Out(A_Gamma) by using the function “check_finite_index”.

As noted above, to make this file readable, we used the function “print_file” from the file "functions.py". Also, we always save the pair (Gamma, Theta) when we find a new example.

In [ ]:
#Small graphs: Computes all Theta graphs for the Gamma graph of fixed size n

n = 5
i_max = 2**(math.comb(n,2)) #Number of (labelled) graphs with n vertices

name_of_output_file = "theta_graphs_gamma_of_size_n="+str(n)+"_all_internal.txt"

graphs = set()
number = 0

for i in range(i_max):
    progressBar_withnumber("Iterations", i, i_max, number)
    G = allgraphs(i,n)
    if(supp_graphs_forest(G)==True):
        T = theta(G)
        already_there = False
        for H in graphs:
            if(nx.is_isomorphic(G,H[0])): #We check for Gamma as we have a bound on the size of Gamma, but not on the size of Theta
                already_there = True
        if(already_there == False):
            graphs.add((G,T)) #Save the pair (Gamma, Theta)
            number += 1
    
progressBar_withnumber("Iterations", i_max, i_max, number)
    
if os.path.isfile(name_of_output_file):
    os.remove(name_of_output_file)
    
with open(name_of_output_file, "wb") as fp:
    pickle.dump(graphs, fp)

The function below randomizes this procedure and tests for i_{max} random graphs Gamma, with |V(Gamma)| being between m_{min} and m_{max}, whether PSO(A_Gamma) is a right-angled Artin group. We only considered Theta graphs with at most n vertices. The reason for these constraints is that one can roughly say that the larger the involved graphs are the longer the computation takes.

In [ ]:
#Random graphs
def random_graphs(n,m_min,m_max,i_max):
    start_time = time.time()

    name_of_output_file = "theta_graphs_size_at_most_"+str(n)+"_internal.txt"

    if os.path.isfile(name_of_output_file):
        with open(name_of_output_file, "rb") as fp:
            graphs = pickle.load(fp)
    else:
        graphs = set()

    number = 0

    for i in range(i_max):
        progressBar_withnumber("Iterations", i, i_max, number)

        #Make random graph with the number of vertices in [m_min,m_max]
        rand_1 = random.randint(m_min,m_max)
        rand_2 = random.randint(m_min,m_max)
        number_of_vertices = max(rand_1,rand_2)
        rand = random.randint(1,2**math.comb(number_of_vertices, 2)-1)
        G = allgraphs(rand,number_of_vertices)


        if(supp_graphs_forest(G)==True):
            if(theta_number_of_vertices(G)<=n):
                T = theta(G)
                already_there = False
                for H in graphs:
                    if(nx.is_isomorphic(G,H[0])):
                        already_there = True
                if(already_there == False):
                    graphs.add((G,T)) #Save the pair (Gamma, Theta)
                    number += 1

    progressBar_withnumber("Iterations", i_max, i_max, number)

    if os.path.isfile(name_of_output_file):
        os.remove(name_of_output_file)

    with open(name_of_output_file, "wb") as fp:
        pickle.dump(graphs, fp)

    print("The programm needed %s seconds." % (round(time.time() - start_time)))

The first command below just runs the function “random_graphs”. The second command additionally displays how much time was used by which function. This was helpful to analyse for which functions it would be worth to try to make them faster.

In [ ]:
random_graphs(15,8,20,1000)
cProfile.run('random_graphs(15,8,20,1000)')

There are two ways to check which graphs we already found as Theta. First, as above, we can check whether we already used the corresponding graph Gamma. In this approach we might save a graph Theta with several graphs Gamma. In order to change from this approach to the one where we check whether we already found the graph Theta, we used the code below. There, for each graph Theta that occurs with several graphs Gamma, we only save the pair with the smallest graph Gamma we found, i.e. the graph Gamma with the lowest number of
vertices. This does not need to be unique. If there are several smallest graphs Gamma with different edges, we just keep one of them.

In [ ]:
#For every Theta graph that occurs in name_of_input_file we save the pair (Theta, Gamma) with a smallest graph Gamma in name_of_output_file.
name_of_input_file = "theta_graphs_size_at_most_15_internal.txt"
name_of_output_file = "theta_graphs_size_at_most_15_unique_gamma_internal.txt"

with open(name_of_input_file, "rb") as fp:
    graphs = pickle.load(fp)
    
graphs_new = set()

for A in graphs:
    already_there = False
    graphs_to_remove = set()
    for B in graphs_new:
        if nx.is_isomorphic(A[1],B[1]):
            if len(A[0]) < len(B[0]):
                graphs_to_remove.add(B)
            else:
                already_there = True
    if already_there == False:
        graphs_new.add(A)
    for C in graphs_to_remove:
        if C in graphs_new:
            graphs_new.remove(C)

with open(name_of_output_file, "wb") as fp:
    pickle.dump(graphs_new, fp)

With the code below, we saved the Theta graphs of size up to n in separate documents, one for each size.

In [ ]:
#Sort graphs in name_of_output_file according to their size.
n = 6

name_of_input_file = "theta_graphs_size_at_most_15_internal.txt"
    
for m in range(n+1):
    name_of_output_file = "theta_graphs_size_equal_to_"+str(m)+"_internal.txt"
    extract_small_theta_graphs(name_of_input_file, name_of_output_file, m)

The code below was used to summarise the results for small Theta graphs. We sorted the graph with the code above and then summarised how many graphs of size up to n have already been found and which ones are missing. This code only works for n at most 7 as we entered the number of graphs for each size between zero and seven manually. These numbers were taken from https://garsia.math.yorku.ca/~zabrocki/math3260w03/nall.html, last accessed on 19th Februrary 2022.

In [ ]:
#Write in name_of_output_file how many graphs of size up to n have been found as Theta
n = 6
name_of_output_file = "Summary_theta_size_up_to_"+str(n)+".txt"

graphs_number = (1,1,2,4,11,34,156,1252)

if os.path.isfile(name_of_output_file):
    os.remove(name_of_output_file)

for m in range(n+1):
    name_of_input_file = "theta_graphs_size_equal_to_"+str(m)+"_internal.txt"
    
    with open(name_of_input_file, "rb") as fp:
        graphs = pickle.load(fp)
    
    out=open(name_of_output_file,'a')
    
    if len(graphs) == graphs_number[m]:
        out.write("All "+str(len(graphs))+" graphs of size "+str(m)+" have been found."+'\n'+'\n'+'\n')
    else:
        out.write(str(len(graphs))+" out of "+str(graphs_number[m])+" graphs of size "+str(m)+" have been found. The graphs that are missing are:"+'\n')
        for G in nx.graph_atlas_g():
            if(len(G.nodes()) == m):
                there = False
                for A in graphs:
                    if(nx.is_isomorphic(G,A[1])):
                        there = True
                if(there == False):
                    out.write("Number of vertices of Theta: "+str(len(G.nodes))+'\n')
                    out.write("Number of edges of Theta: "+str(len(G.edges))+'\n')
                    out.write("Vertices of Theta: " + str(G.nodes)+'\n')
                    out.write("Edges of Theta: " + str(G.edges)+'\n')
                    out.write('\n')
        out.write('\n')
        out.close()

The code below was used to find examples for the special cases that are discussed in the appendix. Note that this code does not check whether the found graphs have no non-trivial graph automorphisms; this was done manually afterwards.

In [ ]:
#We try to find a graph Gamma such that PSO(A_\Gamma) is isomorphic to A_{H_i} for some i and PSO(A_\Gamma) has finite index in Out(A_\Gamma).
H1 = nx.Graph()
H1.add_nodes_from([0])
H1.add_edges_from([])
H2 = nx.Graph()
H2.add_nodes_from([0,1])
H2.add_edges_from([])
H3 = nx.Graph()
H3.add_nodes_from([0,1])
H3.add_edges_from([(0,1)])


#We test Gamma graphs with more than m_min and less than m_max vertices.
n = 2
m_min = 8
m_max = 20
i_max = 1000 #Number of graphs we try


for i in range(i_max):
    progressBar("Iterations", i, i_max)

    #Make random graph with the number of vertices in [m_min,m_max]
    rand_1 = random.randint(m_min,m_max)
    rand_2 = random.randint(m_min,m_max)
    number_of_vertices = max(rand_1,rand_2)
    rand = random.randint(1,2**math.comb(number_of_vertices, 2)-1)
    G = allgraphs(rand,number_of_vertices)


    if(supp_graphs_forest(G)==True):
        if(theta_number_of_vertices(G)<=n):
            T = theta(G)
            if(nx.is_isomorphic(T,H1) or nx.is_isomorphic(T,H2) or nx.is_isomorphic(T,H3)):
                if(check_finite_index(G)):
                    print()
                    print(G.nodes())
                    print(G.edges())

progressBar("Iterations", i_max, i_max)

The following two functions can be used to construct, for a given graph Lambda with at least three vertices, the graphs Gamma respectively Gamma′ that we described in Theorems 3.1 and 3.4. They only work if the graph Lambda that is given as input has vertices {0,...,n-1}.

In [ ]:
#Only works if Gamma has vertices {0,...,n-1}
def construct_gamma(H):
    G = copy.deepcopy(H)
    n = len(H.nodes())
    
    #Add additional vertices
    G.add_nodes_from(["a1","a2","b1","b2","d1","d2"])
    for i in range(n-1):
        G.add_node("c"+str(i))
    
    #Add additional edges
    G.add_edge("a1","b1")
    G.add_edge("a2","b2")
    G.add_edge("a1","a2")
    G.add_edges_from([("b1","d1"),("b2","d2")])
    for i in range(n):
        G.add_edge(i,"b1")
        G.add_edge(i,"b2")
    for i in range(n-1):
        G.add_edges_from([("c"+str(i),i),("c"+str(i),i+1),("c"+str(i),"d1"),("c"+str(i),"d2")])
    G.add_edges_from([("c"+str(n-1),n-1),("c"+str(n-1),0),("c"+str(n-1),"d1"),("c"+str(n-1),"d2")])
    G.add_edge("d1",0)
    for i in range(1,n):
        G.add_edge("d2",i)
    
    return G

In [ ]:
#Only works if Gamma has vertices {0,...,n-1}
def construct_gamma_2(H):
    G = copy.deepcopy(H)
    n = len(H.nodes())
    
    #Add additional vertices
    G.add_nodes_from(["a1","a2","a3","b1","b2","b3","d1","d2","d3"])
    for i in range(n-1):
        G.add_node("c"+str(i))
    
    #Add additional edges
    G.add_edge("a1","b1")
    G.add_edge("a2","b2")
    G.add_edge("a3","b3")
    G.add_edges_from([("a1","a2"),("a1","a3"),("a2","a3")])
    G.add_edges_from([("b1","d1"),("b3","d3")])
    for i in range(n):
        G.add_edge(i,"b1")
        G.add_edge(i,"b2")
        G.add_edge(i,"b3")
    for i in range(n-1):
        G.add_edges_from([("c"+str(i),i),("c"+str(i),i+1),("c"+str(i),"d1"),("c"+str(i),"d2"),("c"+str(i),"d3")])
    G.add_edges_from([("c"+str(n-1),n-1),("c"+str(n-1),0),("c"+str(n-1),"d1"),("c"+str(n-1),"d2"),("c"+str(n-1),"d3")])
    G.add_edges_from([("d1",0),("d2",1)])
    for i in range(2,n):
        G.add_edge("d3",i)
    G.add_edge("d2","d3")
    
    return G

We used the following code to check whether the construction works. This code checks the construction for graphs Lambda of size at most seven and outputs for graphs that do not work how many vertices these graphs have. Running it, one can see that the construction works for all graphs with between three and seven vertices but not for the graphs with at most two vertices. One can do the same for the second construction.

In [ ]:
for H in nx.graph_atlas_g():
    G = construct_gamma(H)
    if supp_graphs_forest(G) == True and check_finite_index(G) == True:
        T = theta(G)
        if nx.is_isomorphic(H,T) == False:
            print('Error: '+str(len(H.nodes()))+' vertex graph')
    else:
        print('Error: '+str(len(H.nodes()))+' vertex graph')

The code below can be used to verify the examples given in Appendix A. For each example, there is one part to check the conditions for Theorem 3.1 and another one to check the additional condition of Theorem 3.4.

For the graph Gamma_1:

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
G.add_edges_from([(1, 2), (1, 4), (1, 5), (2, 3), (2, 6), (2, 8), (3, 4), (3, 6), (3, 8), (4, 9), (4, 10), (4, 11), (5, 6), (5, 9), (5, 10), (6, 7), (6, 9), (7, 8), (7, 9), (7, 11), (8, 9), (8, 10), (10, 11)])
print(supp_graphs_forest(G))
if supp_graphs_forest(G):
    print(check_finite_index(G))
    print("Vertices of Theta: "+str(theta(G).nodes())+", edges of Theta: "+str(theta(G).edges))

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
G.add_edges_from([(1, 2), (1, 4), (1, 5), (2, 3), (2, 6), (2, 8), (3, 4), (3, 6), (3, 8), (4, 9), (4, 10), (4, 11), (5, 6), (5, 9), (5, 10), (6, 7), (6, 9), (7, 8), (7, 9), (7, 11), (8, 9), (8, 10), (10, 11)])

print("Compute the degrees of all vertices:")
for i in range(1,12):
    print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, {1,11} needs to be fixed (not necessarily point-wise).")

print("Check common neighbours of 1 and 11:")
for i in range(1,12):
    if (1,i) in G.edges() and (11,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 4 needs to be fixed.")

print("Check neighbours of 4:")
for i in range(1,12):
    if (4,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 9 needs to be fixed.")
print("Also, {3,10} needs to be fixed (not necessarily point-wise).")

print("Check common neighbours of 3 and 10:")
for i in range(1,12):
    if (3,i) in G.edges() and (10,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 8 needs to be fixed (as 4 is).")

print("Check neighbours of 9:")
for i in range(1,12):
    if (9,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, also 6 needs to be fixed (as 4 and 8 are).")

print("Check common neighbours of 8 and 9:")
for i in range(1,12):
    if (8,i) in G.edges() and (9,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 7 needs to be fixed.")

print("Check neighbours of 7:")
for i in range(1,12):
    if (7,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 11 needs to be fixed.")

print("Check neighbours of 11:")
for i in range(1,12):
    if (11,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 10 needs to be fixed (as 7 is).")
print("So, also 3 needs to be fixed (as {3,10} is).")

print("Check neighbours of 10:")
for i in range(1,12):
    if (10,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 5 needs to be fixed.")

print("Check degrees of remaining vertices:")
for i in [1,2]:
    print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, also 1 and 2 need to be fixed.")
print("Hence, this graph has no non-trivial graph automorphisms.")

For the graph Gamma_2:

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9])
G.add_edges_from([(1, 2), (1, 3), (1, 5), (1, 6), (2, 3), (2, 4), (2, 7), (3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 9), (8, 9)])
print(supp_graphs_forest(G))
if supp_graphs_forest(G):
    print(check_finite_index(G))
    print("Vertices of Theta: "+str(theta(G).nodes())+", edges of Theta: "+str(theta(G).edges))

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9])
G.add_edges_from([(1, 2), (1, 3), (1, 5), (1, 6), (2, 3), (2, 4), (2, 7), (3, 6), (3, 7), (3, 8), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 9), (8, 9)])

print("Compute the degrees of all vertices:")
for i in range(1,10):
    print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, {8,9} needs to be fixed (not necessarily point-wise).")

print("Check common neighbours of 8 and 9:")
for i in range(1,10):
    if (8,i) in G.edges() and (9,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 4 needs to be fixed.")
print("So, also 3 needs to be fixed (as 4 is; see all vertices).")

print("Check neighbours of 3:")
for i in range(1,10):
    if (3,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 8 needs to be fixed.")
print("So, also 9 needs to be fixed (as 8 is; see all vertices).")

print("Check neighbours of 9:")
for i in range(1,10):
    if (9,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 5 needs to be fixed.")

print("Check common neighbours of 4 and 5:")
for i in range(1,10):
    if (4,i) in G.edges() and (5,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, {6,7} needs to be fixed (not necessarily point-wise).")

print("Check neighbours of 4:")
for i in range(1,10):
    if (4,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 2 needs to be fixed (as {6,7} is fixed).")
print("So, also the remaining vertex except {6,7}, which is 1, needs to be fixed.")

print("Check neighbours of 2:")
for i in range(1,10):
    if (2,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 7 needs to be fixed (as 1 is).")
print("So, also the remaining vertex 6 needs to be fixed.")
print("Hence, this graph has no non-trivial graph automorphisms.")

For the graph Gamma_3:

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
G.add_edges_from([(1, 2), (1, 3), (1, 5), (1, 6), (2, 4), (2, 5), (2, 7), (3, 4), (3, 8), (4, 5), (4, 8), (4, 9), (4, 10), (5, 6), (5, 9), (6, 8), (6, 9), (6, 11), (7, 8), (7, 9), (7, 11), (8, 9), (10, 11)])
print(supp_graphs_forest(G))
if supp_graphs_forest(G):
    print(check_finite_index(G))
    print("Vertices of Theta: "+str(theta(G).nodes())+", edges of Theta: "+str(theta(G).edges))

In [ ]:
from functions import *
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
G.add_edges_from([(1, 2), (1, 3), (1, 5), (1, 6), (2, 4), (2, 5), (2, 7), (3, 4), (3, 8), (4, 5), (4, 8), (4, 9), (4, 10), (5, 6), (5, 9), (6, 8), (6, 9), (6, 11), (7, 8), (7, 9), (7, 11), (8, 9), (10, 11)])

print("Compute the degrees of all vertices:")
for i in range(1,12):
    print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 4 and 10 need to be fixed.")

print("Check neighbours of 10:")
for i in range(1,12):
    if (10,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 11 needs to be fixed.")
print("So, also 3 needs to be fixed (as 11 is; see all vertices).")

print("Check neighbours of 11:")
for i in range(1,12):
    if (11,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 6 and 7 need to be fixed.")

print("Check neighbours of 7:")
for i in range(1,12):
    if (7,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 2 needs to be fixed.")

print("Check neighbours of 2:")
for i in range(1,12):
    if (2,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, 1 (as 7 is) and 5 need to be fixed.")

print("Check neighbours of 3:")
for i in range(1,12):
    if (3,i) in G.edges():
        print("     "+str(i)+" has degree "+str(G.degree(i)))
print("So, also 8 needs to be fixed.")
print("So, also the only remaining vertex 9 needs to be fixed.")
print("Hence, this graph has no non-trivial graph automorphisms.")